In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import numpy as np
import os
from train.train import parse_config, get_features
from train.eval import makeRoc, plot_confusion_matrix
from models.constraints import ZeroSomeWeights
from layers.quantized_layers import Clip, BinaryDense, TernaryDense, QuantizedDense
from models.models import binary_tanh, ternary_tanh, quantized_relu
import yaml
from sklearn.metrics import confusion_matrix
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

2025-06-23 08:40:04.856312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


sys.path: ['/home/tim_legge/keras-trainin-legge', '/home/tim_legge/miniconda3/envs/tensorflow/lib/python37.zip', '/home/tim_legge/miniconda3/envs/tensorflow/lib/python3.7', '/home/tim_legge/miniconda3/envs/tensorflow/lib/python3.7/lib-dynload', '', '/home/tim_legge/miniconda3/envs/tensorflow/lib/python3.7/site-packages', '/home/tim_legge/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/extensions', '/home/tim_legge/.ipython']


In [3]:
f = h5py.File('/mnt/c/ML_Data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z')
treeArray = f['t_allpar_new'][()]
print(treeArray.dtype.names)

('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j1_px', 'j1_py', 'j1_pz', 'j1_e', 'j1_pdgid', 'j1_erel', 'j1_pt', 'j1_ptrel', 'j1_eta', 'j1_etarel', 'j1_etarot', 'j1_phi', 'j1_phirel', 'j1_phirot', 'j1_deltaR', 'j1_costheta', 'j1_costhetarel', 'j1_e1mcosthetarel', 'j_index', 'j_g', 'j_q'

In [4]:
#getting features, similar to train.train

with open("train/train_config_threelayer.yml", "r") as file:
    yamlConfig = yaml.safe_load(file)
    
    # List of features to use
features = yamlConfig["Inputs"]

    # List of labels to use
labels = yamlConfig["Labels"]

In [5]:
    # Converts to dataframe
features_labels_df = pd.DataFrame(treeArray, columns=list(set(features + labels)))
features_labels_df = features_labels_df.drop_duplicates()

features_df = features_labels_df[features]
labels_df = features_labels_df[labels]
    
features_val = features_df.values
labels_val = labels_df.values
    
X_train_val, X_test, y_train_val, y_test = train_test_split(
        features_val, labels_val, test_size=0.2, random_state=42)

In [6]:
#normalizing

scaler = preprocessing.StandardScaler().fit(X_train_val)
X_train_val = scaler.transform(X_train_val)
X_test = scaler.transform(X_test)        

In [7]:
import models.models
from keras.layers import Input
print(yamlConfig['LayerSizes'])
model = getattr(models.models, yamlConfig["KerasModel"])
keras_model = model(
            Input(shape=X_train_val.shape[1:]),
            y_train_val.shape[1],
            l1Reg=yamlConfig["L1Reg"],)

[64, 32, 32]


2025-06-23 08:41:58.870269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-23 08:41:58.874181: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
from train.train import print_model_to_json
outputDir = 'train_simple_l10p0001'
print_model_to_json(keras_model, outputDir + "/" + "KERAS_model.json")

In [ ]:
from keras.optimizers import Adam, Nadam
startlearningrate = 0.0001
adam = Adam(learning_rate=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig["KerasLoss"]], metrics=["accuracy"])

In [ ]:
from train.callbacks import all_callbacks
callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir=outputDir,
    )

keras_model.fit(
    X_train_val,
    y_train_val,
    batch_size=1024,
    epochs=100,
    validation_split=0.25,
    shuffle=True,
    callbacks=callbacks.callbacks,
    )

In [9]:
inputModel = 'train_simple_l10p0001/KERAS_check_best_model.h5'

model = load_model(
        inputModel,
        custom_objects={
            "ZeroSomeWeights": ZeroSomeWeights,
            "BinaryDense": BinaryDense,
            "TernaryDense": TernaryDense,
            "QuantizedDense": QuantizedDense,
            "binary_tanh": binary_tanh,
            "ternary_tanh": ternary_tanh,
            "quantized_relu": quantized_relu,
            "Clip": Clip,
        },
    )

In [10]:
outputDir = 'train_simple_l10p0001'
y_predict = makeRoc(X_test, labels, y_test, model, outputDir)

in makeRoc()
617/617 [==============================] - 2s 2ms/step


In [12]:
y_test_proba = y_test.argmax(axis=1)
y_predict_proba = y_predict.argmax(axis=1)
# Compute non-normalized confusion matrix
cnf_matrix = confusion_matrix(y_test_proba, y_predict_proba)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(
    cnf_matrix,
    classes=[l.replace("j_", "") for l in labels],
    title="Confusion matrix",
    )
plt.figtext(
    0.28,
    0.90,
    "hls4ml",
    fontweight="bold",
    wrap=True,
    horizontalalignment="right",
    fontsize=14,
    )
    # plt.figtext(0.38, 0.90,'preliminary', style='italic', wrap=True, horizontalalignment='center', fontsize=14)
plt.savefig(outputDir + "/confusion_matrix.pdf")
    # Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(
    cnf_matrix,
    classes=[l.replace("j_", "") for l in labels],
    normalize=True,
    title="Normalized confusion matrix",
    )

plt.figtext(
    0.28,
    0.90,
    "hls4ml",
    fontweight="bold",
    wrap=True,
    horizontalalignment="right",
    fontsize=14,
    )
    # plt.figtext(0.38, 0.90,'preliminary', style='italic', wrap=True, horizontalalignment='center', fontsize=14)
plt.savefig(outputDir + "/confusion_matrix_norm.pdf")